In [1]:
!pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [10]:
pip install -U bitsandbytes

In [16]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import userdata


try:
    hf_token = userdata.get('hf_token')
    if not hf_token:
        print("Warning: HF_TOKEN not found in Colab Secrets under 'hf_token'. Proceeding for public model access.")
        hf_token = None
except Exception as e:
    print(f"Error retrieving HF_TOKEN: {str(e)}")
    print("Warning: Token retrieval failed, but proceeding for public model access.")
    hf_token = None


!pip install -U transformers accelerate -q


try:
    tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it", token=hf_token)
    model = AutoModelForCausalLM.from_pretrained(
        "google/gemma-2-2b-it",
        token=hf_token,
        device_map="auto",
        torch_dtype=torch.float16
    )
except Exception as e:
    print(f"Error loading model or tokenizer: {str(e)}")
    exit(1)

def generate_questions(intern_profile, job_description):

    prompt = f"""
    You are an expert interview question generator for intern roles. Based on the provided intern profile and job description, generate 5 technical and 5 behavioral interview questions tailored to the candidate's skills and the role's requirements. Ensure technical questions match the candidate’s experience level and the job’s technical needs, and behavioral questions follow the STAR framework (Situation, Task, Action, Result).

    Intern Profile:
    {intern_profile}

    Job Description:
    {job_description}

    Output Format:
    Technical Questions:
    1. [Question]
    2. [Question]
    3. [Question]
    4. [Question]
    5. [Question]

    Behavioral Questions:
    1. [Question]
    2. [Question]
    3. [Question]
    4. [Question]
    5. [Question]
    """
    messages = [
        {"role": "user", "content": prompt}
    ]
    try:

        inputs = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        ).to(model.device)


        outputs = model.generate(
            **inputs,
            max_new_tokens=600,
            temperature=0.7,
            do_sample=True,
            top_p=0.9
        )

        generated_text = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
        return generated_text.strip()
    except Exception as e:
        return f"Error generating questions: {str(e)}"


intern_profile_1 = """
- Role: Data Science Intern
- Skills: Python, SQL, basic machine learning
- Experience: Beginner, 2nd-year BSc Computer Science
- Projects: Built a linear regression model for a university project
"""
job_description_1 = """
- Role: Data Science Intern
- Skills Required: Python, data analysis, team collaboration
- Responsibilities: Analyze datasets, build predictive models, present findings
"""

# Sample Input 2: Software Engineering Intern
intern_profile_2 = """
- Role: Software Engineering Intern
- Skills: Python, Java, basic understanding of web development (HTML, CSS)
- Experience: Beginner, 3rd-year BSc Computer Science
- Projects: Developed a simple web application using Flask and Python for a course project
"""
job_description_2 = """
- Role: Software Engineering Intern
- Skills Required: Python, Java, web development, problem-solving
- Responsibilities: Assist in developing backend features, write clean code, participate in code reviews
"""


intern_profile_3 = """
- Role: Product Management Intern
- Skills: Basic Python, user research, communication
- Experience: Beginner, 2nd-year BA Business Administration
- Projects: Conducted user interviews for a mobile app prototype in a class project
"""
job_description_3 = """
- Role: Product Management Intern
- Skills Required: User research, data analysis, teamwork
- Responsibilities: Gather user feedback, assist in defining product requirements, analyze usage data
"""


sample_inputs = [
    ("Data Science Intern", intern_profile_1, job_description_1),
    ("Software Engineering Intern", intern_profile_2, job_description_2),
    ("Product Management Intern", intern_profile_3, job_description_3)
]

for role, intern_profile, job_description in sample_inputs:
    print(f"\n=== {role} Questions ===")
    questions = generate_questions(intern_profile, job_description)
    print(questions)

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]


=== Data Science Intern Questions ===


W0721 15:51:38.774000 534 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode


## Technical Questions:

1. **Can you walk me through the process of building a linear regression model, starting from data cleaning and feature engineering to model evaluation?** (This assesses their understanding of the ML workflow and ability to articulate it)
2. **You're given a dataset with missing values. How would you handle these missing values in your analysis?** (Tests their knowledge of data preprocessing techniques)
3. **Explain the difference between a classification model and a regression model. Provide an example of when each would be appropriate.** (Evaluates their understanding of different model types and their application scenarios)
4. **Describe a time you encountered a challenging data analysis problem. How did you approach it, and what was the outcome?** (Assesses their problem-solving skills and ability to handle real-world issues)
5. **What are some popular libraries in Python for data visualization? Why are they useful in data science?** (Tests their familiarit